# Final Project for INFO 6350
## Net ID: yz2685

In [1]:
# import our libraries
import requests
import urllib
from bs4 import BeautifulSoup
import simplejson as json
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from selenium import webdriver
import time
import re
import pandas as pd
import json
import datetime

In [2]:
# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
# content = requests.get(documents_url, headers=header)
# print(content)

In [3]:
# four-digit year pattern
yearPattern = re.compile(r'\d{4}$')

In [4]:
list_cik = pd.read_csv('nasdaq100_ticker_cik_mapping.csv').fillna(value = 0).head(3)
list_cik.head()

,symbol,company,cik,market_cap_05132022,last_sale_05132022
0,AAPL,Apple Inc. Common Stock,320193,"2,550,149,340,600",$147.09
1,MSFT,Microsoft Corporation Common Stock,789019,"1,954,720,100,164",$261.36
2,GOOG,Alphabet Inc. Class C Capital Stock,1652044,"1,542,091,474,456","$2,341.83"


In [5]:
len(list_cik)

3

## Scraping the SEC Query Page

In [6]:
# define lists to store the data scraped from the SEC website
ciks = []
risks = []
years = []
urls = []
companies = []
symbols = []

# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

In [7]:
driver = webdriver.Firefox()

### Helper functions 

In [8]:
# get the links to the 10k reports 
def get10kPages(url):
    response = requests.get(url = url, headers=header)
    soup10k = BeautifulSoup(response.content, 'html.parser')
    # print(response)
    # print(response.url)
    return response.url

In [9]:
def get10kLinks(url, list_of_10ks):
    response = requests.get(url = url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    # for a in soup.find_all('a', href=True):
        # url = a['href']
        # print(url)
        
    suffix = "htm";

    tables = soup.find('table')
    rows = tables.find_all('tr')
    if len(rows) > 0:
        row10k = rows[1] # row 1 has link to 10k report
        # print(row10k)
        for a in row10k.find_all('a', href=True):
            url = a['href']
            if url.endswith(suffix):
                list_of_10ks.append("https://www.sec.gov"+ url)
                # print("https://www.sec.gov"+ url)

In [10]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [11]:
# create a function that will scrap the 10k report 
# import chromedriver_binary  # Adds chromedriver binary to path


def scrap10k(url, cik, company, symbol):
    driver.get(url)

    time.sleep(2) # give browser some time to load the js 

    html = driver.page_source
    sp = BeautifulSoup(html)
    
    text = ""
    for d in sp.find_all(text=True):
        text += d.get_text()
    
    # cleaning 
    # print(text)
    text = text.replace(u'\xa0', u' ').lower()
                
    # extract risk factor sections only
    # print(url)
    
    # get fiscal year     
    yr = ""
    for span in sp.find_all(text=True):
        stext = span.text
        if (stext.find("January") != -1
         or stext.find("February") != -1
         or stext.find("March") != -1
         or stext.find("April") != -1
         or stext.find("May") != -1
         or stext.find("June") != -1
         or stext.find("July") != -1
         or stext.find("August") != -1
         or stext.find("September") != -1
         or stext.find("October") != -1
         or stext.find("November") != -1
         or stext.find("December") != -1):
            stext = stext.strip()
            yr = stext[-4:]
            # print(yr)
            break
            
    
    # print("=================")    
    # print(text)
    
    # proceed only when a valid year is scraped 
    yr_match = re.match(yearPattern, yr)
    if yr_match != None:
        yr = int(yr) # convert string to int
        if yr > 2006:
            start = find_nth(text, "item 1a.", 2)
            # print("start index", start)
            end = find_nth(text, "item 1b.", 2)
            # print("end index", end)
            substring = text[start:end]
            if len(substring) > 0:
                risks.append(substring)
                years.append(yr)
                ciks.append(cik)
                urls.append(url)
                companies.append(company)
                symbols.append(symbol)
    #     else:
    #         print("year before 2007: ", yr)
    # else:
    #     print("fiscal year not detected")

In [12]:
def getDataByCIK(cik, company, symbol):
    ########################################
    ### Step 1. Scraping the SEC Query Page
    ########################################
    # define our parameters dictionary
    param_dict = {'action':'getcompany',
                  'CIK': cik,
                  'type':'10-k',
                  'dateb':'20230101',
                  'owner':'exclude',
                  'start':'',
                  'output':'',
                  'count':'100'}

    # request the url, and then parse the response.
    response = requests.get(url = endpoint, params = param_dict, headers=header)
    # response = requests.get(url = endpoint, params = param_dict)
    soup = BeautifulSoup(response.content, 'html.parser')

    # print('Request Successful')
    # print(response.url)
    
    doc_table = soup.find_all(class_ = "blueRow")
    
    data = soup.find_all(class_='blueRow')

    list_10k = []

    for i, row in enumerate(data): 
        for a in data[i].find_all('a', href=True):
            url = a['href']
            if (url.startswith('/Archives/edgar/')):
                list_10k.append("https://www.sec.gov"+ url)
                # print("https://www.sec.gov"+ url)
                
    ########################################
    ### Step 2. Scraping Company Page 
    ########################################
    
    list_of_links = []
    for link in list_10k:
        list_of_links.append(get10kPages(link))
        
    # get the url for 10k report 
    list_of_10ks = []
    for url in list_of_links:
        get10kLinks(url, list_of_10ks)
        

    ########################################
    ### Step 3. Scraping 10k Reports 
    ########################################
    for report in list_of_10ks:
        scrap10k(report, cik, company, symbol)

In [13]:
# test
# scrap10k('https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/amzn-20181231x10k.htm', "1018724", "amzn")

In [14]:
# test
# getDataByCIK("1067983", "brk-b")

### Run scripts for all the companies of interest

In [15]:
%%time
for i, row in list_cik.iterrows():
    getDataByCIK(row.cik, row.company, row.symbol)

CPU times: user 27.6 s, sys: 1.01 s, total: 28.6 s
Wall time: 1min 39s


### Create Data Frame

In [16]:
# create a data frame
data_tuples = list(zip(ciks, symbols, companies, years, risks, urls))
# data_tuples

df = pd.DataFrame(data_tuples, columns=['cik', 'symbol', 'company', 'fiscal_year', 'risk', 'url'])

In [17]:
df.head()

,cik,symbol,company,fiscal_year,risk,url
0,320193,AAPL,Apple Inc. Common Stock,2018,item 1a.risk factorsthe following discussion o...,https://www.sec.gov/Archives/edgar/data/320193...
1,320193,AAPL,Apple Inc. Common Stock,2016,item 1a. risk factorsthe following discussion ...,https://www.sec.gov/Archives/edgar/data/320193...
2,320193,AAPL,Apple Inc. Common Stock,2014,item 1a.\nrisk factors the following discussi...,https://www.sec.gov/Archives/edgar/data/320193...
3,320193,AAPL,Apple Inc. Common Stock,2007,item 1a. risk factors\n,https://www.sec.gov/Archives/edgar/data/320193...
4,789019,MSFT,Microsoft Corporation Common Stock,2018,item 1a. risk factors \nour operations and fin...,https://www.sec.gov/Archives/edgar/data/789019...


In [18]:
len(df)

10

In [27]:
df

,cik,symbol,company,fiscal_year,risk,url
0,320193,AAPL,Apple Inc. Common Stock,2018,item 1a.risk factorsthe following discussion o...,https://www.sec.gov/Archives/edgar/data/320193...
1,320193,AAPL,Apple Inc. Common Stock,2016,item 1a. risk factorsthe following discussion ...,https://www.sec.gov/Archives/edgar/data/320193...
2,320193,AAPL,Apple Inc. Common Stock,2014,item 1a.\nrisk factors the following discussi...,https://www.sec.gov/Archives/edgar/data/320193...
3,320193,AAPL,Apple Inc. Common Stock,2007,item 1a. risk factors\n,https://www.sec.gov/Archives/edgar/data/320193...
4,789019,MSFT,Microsoft Corporation Common Stock,2018,item 1a. risk factors \nour operations and fin...,https://www.sec.gov/Archives/edgar/data/789019...
5,789019,MSFT,Microsoft Corporation Common Stock,2016,item 1a. risk factors our operations and fina...,https://www.sec.gov/Archives/edgar/data/789019...
6,789019,MSFT,Microsoft Corporation Common Stock,2014,item 1a. risk factors our operations and fina...,https://www.sec.gov/Archives/edgar/data/789019...
7,789019,MSFT,Microsoft Corporation Common Stock,2012,item 1a. risk factors our operations and fina...,https://www.sec.gov/Archives/edgar/data/789019...
8,789019,MSFT,Microsoft Corporation Common Stock,2010,item 1a. risk factors \nour operations and fin...,https://www.sec.gov/Archives/edgar/data/789019...
9,1652044,GOOG,Alphabet Inc. Class C Capital Stock,2017,item 1a.risk factorsour operations and financi...,https://www.sec.gov/Archives/edgar/data/165204...


## Scrap stock prices from Yahoo Finance 

In [204]:
# get parameters 

# ticker = 'AMZN'
# period1 = '1167609600' # 2007-01-01
# period2 = '1672444800' # 2022-12-30

dict = {'symbol': [], 'dates': [], 'prices': []}

In [217]:
def getStockPricesByTicker(ticker, period1, period2):
    url_history = f'https://finance.yahoo.com/quote/{ticker}/history?period1={period1}&period2={period2}&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=false'
    
    respf = requests.get(url_history, headers=header)
    print(respf)
    
    soupf = BeautifulSoup(respf.text, 'html.parser')
    
    patternf = re.compile(r'\s--\sData\s--\s')
    script_data = soupf.find('script', text=patternf).contents[0]
    
    start = script_data.find("context")-2
    json_data = json.loads(script_data[start:-12])
    
    # get historical stock prices 
    HistoricalPriceStore = json_data['context']['dispatcher']['stores']['HistoricalPriceStore']
    # HistoricalPriceStore.keys()
    
    # print(HistoricalPriceStore)
    
    # clean the stock price data 

    for row in HistoricalPriceStore['prices']:
        date = row['date']
        dt_formatted = datetime.datetime.fromtimestamp(date).date() # using the local timezone

        # get the stock price in December of a given year 
        if dt_formatted.month == 12:
            dict['symbol'].append(ticker)
            dict['dates'].append(dt_formatted)
            dict['prices'].append(row['close'])

In [218]:
# test
getStockPricesByTicker('AMZN', '1167609600', '1672444800')

<Response [200]>


In [ ]:
# %%time
# for i, row in list_cik.iterrows():
#     getDataByCIK(row.cik, row.company, row.symbol)

In [220]:
dfItem = pd.DataFrame.from_records(dict)
dfItem

,dates,prices,symbol
0,2021-12-01,3334.340088,AMZN
1,2020-12-01,3256.929932,AMZN
2,2019-12-01,1847.839966,AMZN
3,2018-12-01,1501.969971,AMZN
4,2017-12-01,1169.469971,AMZN
5,2016-12-01,749.869995,AMZN
6,2015-12-01,675.890015,AMZN
7,2014-12-01,310.350006,AMZN
8,2013-12-01,398.790009,AMZN
9,2012-12-01,250.869995,AMZN


## Data Cleaning

# Analysis

In [20]:
# import unicodedata

# text = unicodedata.normalize('NFKD', text.replace("\'", "'").replace("\ in\ form", " inform").replace("\n", " ").lower().strip())

In [21]:
# print(text)
# dev_docs = [doc for doc in text.split('\n')] 

In [22]:
# from   sklearn.feature_extraction.text import TfidfVectorizer
# from   nltk.corpus import stopwords
# import string

# # Making stopwords list
# stoplist = stopwords.words('english')
# for el in [i for i in string.punctuation]:
#     stoplist.append(el)

# # Set up vectorizer
# vectorizer = TfidfVectorizer(
#     encoding='utf-8',
#     min_df=1, # include words that occur in as few as a single document, 1 = a single document
#     max_df=1.0, # include words that occur in as many as all documents, 1.0 = 100% = all documents
#      binary=False,
#     norm='l2',
#     stop_words = stoplist,
#     use_idf=True
# )

# X = vectorizer.fit_transform(dev_docs)

In [23]:
# print(X.shape)

In [24]:
# print(X)

References

https://stackoverflow.com/questions/48687857/python-json-list-to-pandas-dataframe